# **1. Import Library**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import itertools
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# **2. Input Dataset**

In [2]:
path = "C:/Users/User/Videos/Project Management and Machine Learning/Dataset/Time/KONS"

# **3.Clear Excel Output** 

In [3]:
def clear_all_data(file):
    workbook = load_workbook(file)
    sheet_names = workbook.sheetnames
    
    if not sheet_names:
        # If there are no sheets, create a new sheet and make it visible
        workbook.create_sheet("Sheet1")
    else:
        # Remove all sheets except the first one
        for sheet_name in sheet_names[1:]:
            workbook.remove(workbook[sheet_name])
    
    workbook.save(file)
    workbook.close()

In [4]:
clear_path = "C:/Users/User/Videos/Project Management and Machine Learning/Ensemble/KONS/Time"

In [5]:
excel_file1 = clear_path+"/RESULT-ENSEMBLE-KONS-TIME.xlsx"
excel_file2 = clear_path+"/PARAM-ENSEMBLE-KONS-TIME.xlsx"

clear_all_data(excel_file1)
clear_all_data(excel_file2)

# **3. Modeling**

In [6]:
class BlendingEnsemble():
    def __init__(self, alpha=0.1, alpha_r=0.1, degree=1):
        self.models = []
        self.blender = None
        self.alpha = alpha
        self.alpha_r = alpha_r
        self.degree = degree

    def get_models(self):
        models = []
        models.append(('Lasso', Lasso(alpha = self.alpha)))
        models.append(('Ridge', Ridge(alpha = self.alpha_r)))
        models.append(('Poly', make_pipeline(PolynomialFeatures(degree = self.degree),LinearRegression())))
        return models

    def fit(self, X_train, y_train):
        self.models = self.get_models()

        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=1)

        meta_X = []
        for name, model in self.models:            
            model.fit(X_train, y_train)
            yhat = model.predict(X_val)
            yhat = yhat.reshape(-1, 1)
            meta_X.append(yhat)
            
        meta_X = np.hstack(meta_X)

        blender = make_pipeline(PolynomialFeatures(),LinearRegression())

        blender.fit(meta_X, y_val)
        self.blender = blender

        return self.blender

    def predict(self, X_test):
        meta_X = []
        for _, model in self.models:
            yhat = model.predict(X_test)
            yhat = yhat.reshape(-1, 1)
            meta_X.append(yhat)

        meta_X = np.hstack(meta_X)

        return self.blender.predict(meta_X)

    def evaluate_r2(self, y_true, y_pred):
        return r2_score(y_true, y_pred)

    def evaluate_rmse(self, y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))
    def __call__(self, X):
        return self.predict(X)

In [7]:
class EnsembleRegressor:
    def __init__(self, data_source_file, result_file,params_file):
        self.data_source_file = data_source_file
        self.result_file = result_file
        self.params_file = params_file
        self.models = {
            'XGBoost': xgb.XGBRegressor,
            'RandomForest': RandomForestRegressor
        }
    

    def train_and_predict(self, sheet, model_name='Blending', param_grid=None):
        # Splitting features and label
        data = pd.read_excel(self.data_source_file, sheet_name=sheet)
        X = data.drop(columns='Earned Value')
        y = data['Earned Value']

        # Splitting data into training and testing sets
        x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=58)
        x_train = x_train.sort_index(ascending=True)
        y_train = y_train.sort_index(ascending=True)
        x_test = x_test.sort_index(ascending=True)
        y_test = y_test.sort_index(ascending=True)

        # Model selection
        model_class = self.models.get(model_name)
        if model_class is None:
            raise ValueError(f"Invalid model name: {model_name}")

        # Parameter tuning
        best_params, results = self.tune_parameters(model_class, x_train, y_train, param_grid)

        # Train the model with the best parameters
        model = model_class(**best_params)
        model.fit(x_train, y_train)
        
        # Predict on the test set
        x_test.reset_index(drop=True, inplace=True)

        ev_pred = model.predict(x_test)

        # Create a DataFrame with predictions
        perform = pd.DataFrame({'AT':x_test.iloc[:,0].values,'Planned Value':x_test.iloc[:,1].values,
                                'Actual': y_test.values, 'EV_Pred': ev_pred, 'Model': model_name})
        results ['Model']=model_name
        results ['Method'] = "ensemble"
        results ['Subwork'] = sheet 
        results ['Work'] = "KONS"
        results ['Process'] = "Time"
        
        perform ['Method'] = "ensemble"
        perform ['Subwork'] = sheet 
        perform ['Work'] = "KONS"
        perform ['Process'] = "Time"
        # Save the results to Excel files
        self.to_excel(perform, self.result_file, sheet)
        self.to_excel(results, self.params_file, sheet)

    def tune_parameters(self, model_class, X, y, param_grid=None):
        if param_grid is None:
            param_grid = {}  

        # Grid search to find the best parameters
        best_params = None
        best_score = float('inf')
        results = []

        for params in self.grid_search(param_grid):
            model = model_class(**params)
            model.fit(X, y)
            y_pred = model.predict(X)
            r2 = self.evaluate_r2(y, y_pred)
            rmse = self.evaluate_rmse(y, y_pred)

            results.append({**params, 'R2': r2, 'RMSE': rmse})

            if rmse < best_score:
                best_score = rmse
                best_params = params

        results_df = pd.DataFrame(results)
        return best_params, results_df

    @staticmethod
    def grid_search(param_grid):
        keys, values = zip(*param_grid.items())
        for combination in itertools.product(*values):
            yield dict(zip(keys, combination))

    @staticmethod
    def evaluate_r2(y_true, y_pred):
        return r2_score(y_true, y_pred)

    @staticmethod
    def evaluate_rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))
    
    @staticmethod
    def to_excel(df, file, sheet_name):
        try:
            book = load_workbook(file)
            writer = pd.ExcelWriter(file, engine='openpyxl')
            writer.book = book
            writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

            if sheet_name in writer.sheets:
                sheet = writer.sheets[sheet_name]
                last_row = sheet.max_row
            else:
                last_row = 0

            if last_row < 1:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=True, startrow=last_row)

            writer.save()
        except FileNotFoundError:
            df.to_excel(file, sheet_name=sheet_name, index=False)

In [8]:
def blending_ensemble (x):
    Sheet = x
    Data = pd.read_excel (path+"/Dataset.xlsx",sheet_name = Sheet )
    X = Data.drop (columns = 'Earned Value')
    y = Data.drop (columns = X.columns)

    #Splitting Data Training and Testing
    x_train, x_test, y_train, y_test = train_test_split(X, y ,train_size = 0.8,random_state = 58)
    x_train = x_train.sort_index(ascending=True)
    y_train = y_train.sort_index(ascending=True)
    x_test = x_test.sort_index(ascending=True)
    y_test = y_test.sort_index(ascending=True)

    Val =[]
    blg_model = BlendingEnsemble()
    for alpha in np.arange (0.025,0.75,0.25):
        for alpha_r in np.arange(0.25,0.76,0.25) :
            for degree in range (2,5) : 
                    blg_model.alpha = alpha
                    blg_model.alpha_r = alpha_r
                    blg_model.degree = degree
                        ##create the model 

                    blg_model.fit (x_train,y_train)
                        #######Train################
                    y_train_Pred = blg_model.predict (x_train)
                    y_train_Pred = pd.DataFrame({'Y_train_Pred':y_train_Pred.ravel()})

                    y_train.reset_index(drop=True,inplace=True)
                    y_train_Pred.reset_index(drop = True,inplace=True)

                    Compare_train = pd.concat([y_train,y_train_Pred],axis=1)

                    R2_train = r2_score (Compare_train['Earned Value'],Compare_train['Y_train_Pred'])
                    RMSE_train = np.sqrt(mean_squared_error(Compare_train['Earned Value'],Compare_train['Y_train_Pred']))

                        ######Prediction############
                    y_pred = blg_model.predict(x_test)
                    y_pred = pd.DataFrame({'Y_pred':y_pred.ravel()})
                        #reset index for handling NaN
                    y_pred.reset_index(drop=True, inplace=True)
                    y_test.reset_index(drop=True, inplace=True)
                        #Comparing prediction and label data
                    Compare = pd.concat([y_test,y_pred],axis =1)
                        #Performance
                    R2 = r2_score (Compare['Earned Value'],Compare['Y_pred'])
                    RMSE = np.sqrt(mean_squared_error(Compare['Earned Value'],Compare['Y_pred']))
                    Gap = abs (RMSE_train - RMSE)
                    Blending = "Blending"
                    Val.append([alpha, alpha_r,degree, R2, RMSE,Blending ])
                    Value=pd.DataFrame(Val,columns=["alpha","alpha_r","degree","R2","RMSE","Model"])
                    Value ['Method'] = "ensemble"
                    Value ['Subwork'] = Sheet 
                    Value ['Work'] = "KONS"
                    Value ['Process'] = "Time"


    param = Value.sort_values(by = ['RMSE'],ascending = True )
    alpha_param = param['alpha'].iloc[0]
    alpha_r_param = param['alpha_r'].iloc[0]
    degree_param = param['degree'].iloc[0]


    blg_param = BlendingEnsemble(alpha= alpha_param,
                                      alpha_r = alpha_r_param, 
                                      degree = degree_param)
    blg_param.fit(x_train,y_train)
    EV_pred = blg_param.predict(x_test)
    EV_pred = pd.DataFrame({'EV_Pred':EV_pred.ravel()})

    EV_pred.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    x_test.reset_index(drop=True, inplace=True)

    Perform = pd.concat([x_test,y_test,EV_pred],axis=1)
    Perform ['Model'] = Blending
    Perform ['Method'] = "ensemble"
    Perform ['Subwork'] = Sheet 
    Perform ['Work'] = "KONS"
    Perform ['Process'] = "Time"
    
    def to_excel(df, file, sheet_name):
        try:
            book = load_workbook(file)
            writer = pd.ExcelWriter(file, engine='openpyxl')
            writer.book = book
            writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

            if sheet_name in writer.sheets:
                sheet = writer.sheets[sheet_name]
                last_row = sheet.max_row
            else:
                last_row = 0

            if last_row < 1:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=True, startrow=last_row)

            writer.save()
        except FileNotFoundError:
            df.to_excel(file, sheet_name=sheet_name, index=False)
    
    #Converting Value Adjusting All Parameter to Excel
    Adjusting = to_excel(Value,"PARAM-ENSEMBLE-KONS-TIME.xlsx",x)
    Prediction = to_excel(Perform,"RESULT-ENSEMBLE-KONS-TIME.xlsx",x)
    
    return Adjusting, Prediction

# **Running K1B1**

In [9]:
data_source_file = path+"/Dataset.xlsx"
result_file = "RESULT-ENSEMBLE-KONS-TIME.xlsx"
params_file = "PARAM-ENSEMBLE-KONS-TIME.xlsx"

ensemble = EnsembleRegressor(data_source_file, result_file,params_file )


param_grid_xg = { 'n_estimators': [50, 100, 200],'max_depth': [3, 5, 7],'learning_rate': [0.1, 0.05, 0.01] }
param_grid_rf = { 'n_estimators': [50, 100, 200],'max_depth': [3, 5, 7],'min_samples_split': [0.1, 0.05, 0.01] }

In [10]:
ensemble.train_and_predict('K1B1', model_name='XGBoost', param_grid=param_grid_xg)

In [11]:
ensemble.train_and_predict('K1B1', model_name='RandomForest', param_grid=param_grid_rf)

In [12]:
blending_ensemble ('K1B1')

(None, None)

# **Running K1B2**

In [13]:
ensemble.train_and_predict('K1B2', model_name='XGBoost', param_grid=param_grid_xg)

In [14]:
ensemble.train_and_predict('K1B2', model_name='RandomForest', param_grid=param_grid_rf)

In [15]:
blending_ensemble ('K1B2')

(None, None)

# **Running K2B1**

In [16]:
ensemble.train_and_predict('K2B1', model_name='XGBoost', param_grid=param_grid_xg)

In [17]:
ensemble.train_and_predict('K2B1', model_name='RandomForest', param_grid=param_grid_rf)

In [18]:
blending_ensemble ('K2B1')

(None, None)

# **Running K2B2**

In [19]:
ensemble.train_and_predict('K2B2', model_name='XGBoost', param_grid=param_grid_xg)

In [20]:
ensemble.train_and_predict('K2B2', model_name='RandomForest', param_grid=param_grid_rf)

In [21]:
blending_ensemble ('K2B2')

(None, None)

# **Running K3B1**

In [22]:
ensemble.train_and_predict('K3B1', model_name='XGBoost', param_grid=param_grid_xg)

In [23]:
ensemble.train_and_predict('K3B1', model_name='RandomForest', param_grid=param_grid_rf)

In [24]:
blending_ensemble ('K3B1')

(None, None)

# **Running K3B2**

In [25]:
ensemble.train_and_predict('K3B2', model_name='XGBoost', param_grid=param_grid_xg)

In [26]:
ensemble.train_and_predict('K3B2', model_name='RandomForest', param_grid=param_grid_rf)

In [27]:
blending_ensemble ('K3B2')

(None, None)

# **Running KUM**

In [28]:
ensemble.train_and_predict('Kum', model_name='XGBoost', param_grid=param_grid_xg)

In [29]:
ensemble.train_and_predict('Kum', model_name='RandomForest', param_grid=param_grid_rf)

In [30]:
blending_ensemble ('Kum')

(None, None)